# LangChain LCEL

### **학습 목표:** LangChain의 LCEL을 사용하여 LLM 체인을 구성한다.

---

## LCEL(LangChain Expression Language) 

- **LCEL**은 `|` 연산자를 사용해 컴포넌트들을 순차적으로 연결하는 선언적 체이닝을 지원합니다

- **재사용성**이 높아 정의된 체인을 다른 체인의 컴포넌트로 활용할 수 있습니다

- **다양한 실행 방식**(.invoke(), .batch(), .stream(), .astream())으로 동기/비동기 처리가 가능합니다

- **배치 처리**시 자동 최적화를 통해 효율적인 작업 수행이 가능합니다


### 1. 환경 설정

- langsmith 가입 필요 (https://www.langchain.com/langsmith)
- 부분 무료

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGCHAIN_TRACING'))

### 2. LCEL  

#### 1) **Prompt + LLM**

* 기본 구조: `Prompt | LLM` 형태로, 파이프(|) 연산자를 사용해 프롬프트와 LLM을 순차적으로 연결합니다.

* 데이터 흐름: 사용자 입력이 Prompt 템플릿을 통해 처리된 후, LLM에 전달되어 최종 응답이 생성됩니다.

* 실행 순서: 파이프라인은 왼쪽에서 오른쪽으로 순차적으로 실행되며, 각 컴포넌트의 출력이 다음 컴포넌트의 입력으로 전달됩니다.

In [ ]:
from langchain_openai import ChatOpenAI

# LLM model
llm = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.3
    )

# 모델에 프롬프트를 입력
response = llm.invoke("탄소의 원자 번호는 무엇인가요?")

In [ ]:
# 응답 객체 확인
response

In [ ]:
# 응답 객체의 텍스트를 확인
response.content

In [ ]:
# 응답 객체의 메타데이터를 확인
response.response_metadata

In [ ]:
from langchain_core.prompts import PromptTemplate

# 템플릿 문자열 정의
template = """
당신은 {topic} 분야의 전문가입니다. {topic}에 관한 다음 질문에 답변해주세요.
질문: {question}
답변: """

# PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# 템플릿 사용 예시
formatted_prompt = prompt.format(
    topic="인공지능",
    question="딥러닝과 머신러닝의 주요 차이점은 무엇인가요?"
)

print("템플릿 변수:")
print(f"- 필수 변수: {prompt.input_variables}")
print("\n생성된 프롬프트:")
print(formatted_prompt)

In [ ]:
# prompt 객체의 템플릿을 확인
print(prompt.template)

In [8]:
# chain을 구성
chain = prompt | llm

In [ ]:
# chain 객체 확인
chain

In [ ]:
# chain 객체의 입력 스키마를 확인
chain.input_schema.model_json_schema() 

In [11]:
# chain 실행
response = chain.invoke( 
    {
        "topic": "화학(Chemistry)", 
        "question": "탄소의 원자 번호는 무엇인가요?"
    }
)

In [ ]:
# 응답 객체를 출력
response

In [ ]:
# 응답 객체의 텍스트를 출력
response.content

In [ ]:
# 응답 객체의 메타데이터를 출력
response.response_metadata

#### **2) Prompt + LLM + Output Parser**

* 데이터 파이프라인: `Prompt | LLM | OutputParser` 형태로 구성되어 LLM의 출력을 구조화된 형식으로 변환합니다.

* Parser 종류: JSON, XML 등 다양한 형식의 파서를 지원하여 LLM 출력을 원하는 데이터 구조로 변환할 수 있습니다.

* 유효성 검증: Parser가 출력 형식을 검증하여 잘못된 형식의 응답을 필터링하고 안정적인 데이터 처리를 보장합니다.

In [ ]:
### 문자열 출력 파서 

from langchain_core.output_parsers import StrOutputParser

# 출력 파서를 생성
output_parser = StrOutputParser()

# 출력 파서를 실행
output_parser.invoke(response)

In [ ]:
# 출력 파서를 chain에 추가
chain = prompt | llm | output_parser

# chain을 실행
response = chain.invoke(
    {
        "topic": "화학(Chemistry)", 
        "question": "탄소의 원자 번호는 무엇인가요?"
    }
)

# 응답 객체를 출력
response

In [ ]:
# input_schema (chain)
chain.input_schema.model_json_schema()

In [ ]:
# input_schema (prompt)
prompt.input_schema.model_json_schema()

### 3. Runnable

* 실행 인터페이스: 모든 LangChain 컴포넌트는 Runnable 인터페이스를 구현하여 일관된 방식으로 실행됩니다.

* 실행 메서드: `.invoke()`, `.batch()`, `.stream()`, `.astream()` 등 다양한 실행 방식을 제공합니다.

* 호환성: 모든 Runnable 컴포넌트는 파이프(|) 연산자를 통해 연결 가능하며, 재사용이 용이합니다.

* Runnable의 주요 유형:

    * `RunnableSequence`: 여러 Runnable을 순차적으로 실행
    * `RunnablePassthrough`: 입력을 그대로 다음 단계로 전달    
    * `RunnableParallel`: 여러 Runnable을 병렬로 실행
    * `RunnableLambda`: 파이썬 함수를 Runnable로 래핑하여 체인에서 사용

#### 1) **RunnableSequence**

- **RunnableSequence**는 컴포넌트들을 연결하여 순차적으로 데이터를 처리하는 체인입니다

- `|` 연산자로 연결된 각 단계의 **출력이 다음 단계의 입력**으로 전달됩니다

- **다양한 실행 방식**(동기/비동기, 배치/스트리밍)을 지원합니다

- LLM 체인, 데이터 파이프라인, 자동화된 작업 등 **다단계 처리**에 활용됩니다

In [ ]:
from langchain_core.runnables import RunnableSequence
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


# 컴포넌트 정의
prompt = PromptTemplate.from_template("'{text}'를 영어로 번역해주세요. 번역된 문장만을 출력해주세요.")
translator = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)
output_parser = StrOutputParser()

# RunnableSequence 생성 - 함수 사용 
translation_chain = RunnableSequence(
    first=prompt,
    middle=[translator],   # 리스트로 전달하는 점에 주의
    last=output_parser
)

# RunnableSequence 생성 - 연산자 사용
# translation_chain = prompt | translator | output_parser

In [ ]:
# 동기 실행
result = translation_chain.invoke({"text": "안녕하세요"})
print(result)  

**[비동기 처리의 주요 이점]**

* 응답성: I/O 작업 중 다른 작업 수행이 가능하여 시스템 전체 응답성 향상

* 확장성: 여러 요청을 동시에 처리할 수 있어 처리량 증가

* 리소스 활용: CPU와 메모리를 효율적으로 활용하여 성능 최적화

In [22]:
import asyncio
import nest_asyncio

# Jupyter에서 비동기 실행을 위한 설정
nest_asyncio.apply()

# 비동기 실행 함수를 정의
async def run():
    result = await translation_chain.ainvoke({"text": "감사합니다"})
    print(result)

In [ ]:
#  비동기 실행 함수를 호출
await run()

#### 2) **RunnableParallel**

- **RunnableParallel**은 여러 컴포넌트를 딕셔너리 형태로 구성해 **동시 실행**합니다

- 동일한 입력이 모든 병렬 컴포넌트에 전달되며, 결과는 **키-값 쌍**으로 반환됩니다

- **데이터 변환**과 **파이프라인 구성**에 특화되어 있으며, 출력 형식을 다음 단계에 맞게 조정할 수 있습니다

`(1) 질문 분석 체인`

In [ ]:
# 질문과 관련된 분야를 찾는 프롬프트
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


# 출력 파서 정의
output_parser = StrOutputParser()

# 질문 템플릿 정의
question_template = """
다음 카테고리 중 하나로 입력을 분류하세요:
- 화학(Chemistry)
- 물리(Physics)
- 생물(Biology)

# 예시:
입력: 사람의 염색체는 모두 몇개가 있나요?
답변: 생물(Biology)

입력: {question}
답변: """

# 프롬프트 생성
question_prompt = ChatPromptTemplate.from_template(question_template)

# 모델 설정
llm = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.3
    )

# 체인 구성
question_chain = question_prompt | llm | output_parser

# 체인 실행
result = question_chain.invoke({"question": "탄소의 원자 번호는 무엇인가요?"})
print(f"분류 결과: {result}")

분류 결과: 답변: 화학(Chemistry)


`(2) 언어 감지 체인`

In [4]:
# 질문에 사용된 언어를 구분하는 프롬프트
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 출력 파서 정의
output_parser = StrOutputParser()

# 언어 분류 템플릿 정의
language_template = """
입력된 텍스트의 언어를 다음 카테고리 중 하나로 분류하세요:
- 영어(English)
- 한국어(Korean)
- 기타(Others)

# 예시:
입력: How many protons are in a carbon atom?
답변: English

입력: {question}
답변: """

# 프롬프트 생성
language_prompt = ChatPromptTemplate.from_template(language_template)

# 체인 구성
language_chain = language_prompt | llm | output_parser

# 체인 실행 예시
examples = [
    "What is the atomic number of carbon?",
    "탄소의 원자 번호는 무엇인가요?",
    "¿Cuál es el número atómico del carbono?"
]

for example in examples:
    result = language_chain.invoke({"question": example})
    print(f"입력: {example}")
    print(f"분류 결과: {result}\n")

입력: What is the atomic number of carbon?
분류 결과: English

입력: 탄소의 원자 번호는 무엇인가요?
분류 결과: 한국어(Korean)

입력: ¿Cuál es el número atómico del carbono?
분류 결과: 답변: Others



`(3) RunnableParallel을 사용한 병렬 실행 체인`

In [5]:
# 질문과 관련된 분야를 찾아서 질문에 대한 답변을 생성하는 프롬프트
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# 답변 템플릿 정의
answer_template = """
당신은 {topic} 분야의 전문가입니다. {topic}에 관한 질문에 {language}로 답변해주세요.
질문: {question}
답변: """

# 프롬프트 및 체인 구성
answer_prompt = ChatPromptTemplate.from_template(answer_template)
output_parser = StrOutputParser()

# 병렬 처리 체인 구성
answer_chain = RunnableParallel({
    "topic": question_chain,            # 주제 분류 체인
    "language": language_chain,         # 언어 감지 체인
    "question": itemgetter("question")  # 원본 질문 추출
}) | answer_prompt | llm | output_parser

# 체인 실행 예시
result = answer_chain.invoke({
    "question": "탄소의 원자 번호는 무엇인가요?"
})

print("처리 결과:")
print(f"답변: {result}")

처리 결과:
답변: 탄소의 원자 번호는 6입니다.


#### 3) **RunnablePassthrough**

- **RunnablePassthrough**는 입력값을 그대로 전달하여 원본 데이터를 보존합니다

- **RunnableParallel**과 함께 사용되어 입력 데이터를 새로운 키로 매핑할 수 있습니다

- **투명한 데이터 흐름**으로 파이프라인 디버깅과 구성이 용이합니다

In [6]:
from langchain_core.runnables import RunnablePassthrough
import re

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: int(re.search(r'\d+', x["query"]).group()),
)

runnable.invoke({"query": '탄소의 원자 번호는 6입니다.'})

{'passed': {'query': '탄소의 원자 번호는 6입니다.'}, 'modified': 6}

In [ ]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: int(re.search(r'\d+', x).group()),
)

runnable.invoke('탄소의 원자 번호는 6입니다.')

#### 4) **RunnableLambda**

- **RunnableLambda**는 일반 함수를 Runnable 객체로 변환하는 래퍼 컴포넌트입니다

- 체인에 **커스텀 로직**을 쉽게 통합할 수 있어 데이터 전처리, 후처리에 유용합니다

- `|` 연산자로 다른 컴포넌트들과 연결해 **복잡한 처리 흐름**을 구성할 수 있습니다

In [7]:
import re
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# 텍스트에서 숫자를 추출하는 함수
def extract_number(query):
    return int(re.search(r'\d+', query).group())

# RunnablePassthrough로 입력을 그대로 전달하고, RunnableLambda로 숫자 추출 함수 실행
runnable = RunnablePassthrough() | RunnableLambda(extract_number)

# 입력 텍스트에서 6을 추출
result = runnable.invoke('탄소의 원자 번호는 6입니다.')
print(result)  # 출력: 6

6


In [ ]:
from langchain.schema.runnable import RunnableLambda
from langchain.prompts import ChatPromptTemplate

# 데이터 전처리 함수 정의
def preprocess_text(text: str) -> str:
    """ 입력 텍스트를 소문자로 변환하고 양쪽 공백을 제거합니다. """
    return text.strip().lower()

# 후처리 함수 정의
def postprocess_response(response: str) -> dict:
    """ 응답 텍스트를 대문자로 변환하고 길이를 계산합니다. """
    response_text = response.content
    return {
        "processed_response": response_text.upper(),
        "length": len(response_text)
    }

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template("다음 주제에 대해 영어 한 문장으로 설명해주세요: {topic}")

# 처리 파이프라인 구성
chain = (
    RunnableLambda(preprocess_text) |  # 입력 전처리
    prompt |                           # 프롬프트 포맷팅
    llm |                              # LLM 추론
    RunnableLambda(postprocess_response)  # 출력 후처리
)

# 체인 실행
result = chain.invoke("인공지능")
print(f"처리된 응답: {result['processed_response']}")
print(f"응답 길이: {result['length']}")

# [실습]

- **다음과 같은 요구사항을 LCEL로 구현합니다**
   1. 사용자 입력을 받아 요약하기
   2. 요약된 내용을 기반으로 감정 분석하기 (긍정, 부정, 중립)
   3. 요약된 문장과 감정 분석 결과를 출력하기 

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿 정의
summarize_prompt = None

sentiment_prompt = None

# 문자열 출력 파서
output_parser = None

# 체인 구성
model = None

# 요약 체인
summarize_chain = None

# 감정 분석 체인
sentiment_chain = None

# 전체 체인
chain = None

# 사용 예시
text = """오늘 시험에서 만점을 받았다. 
시험 준비를 열심히 해서 결과가 좋았다.
시험이 끝난 후 친구들과 함께 축하 파티를 열었다.
시험이 끝나고 나니 마음이 홀가분하다.
시험 준비를 하면서 힘든 순간도 있었지만,
그런 순간들이 나를 더 강하게 만들어 주었다.
정말 기쁘고 노력한 보람이 있었다!"""
result = None

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

# 모델 & 파서
llm = ChatOpenAI()
parser = StrOutputParser()

# 1. 요약 프롬프트
summarize_prompt = PromptTemplate.from_template("""
다음 글을 한 문장으로 요약해줘:
{text}
""")
summarize_chain = summarize_prompt | llm | parser

# 2. 감정 분석 프롬프트
sentiment_prompt = PromptTemplate.from_template("""
다음 글의 감정을 분석해줘. 감정은 긍정, 부정, 중립 중 하나로 답변해:
{text}
""")
sentiment_chain = sentiment_prompt | llm | parser

# 3. 요약 → 감정 분석 입력으로 활용하도록 병렬 처리 체인 구성
summary_and_sentiment_chain = summarize_chain | RunnableParallel({
    "summary": lambda summary: summary,  # 그냥 넘김
    "sentiment": lambda summary: sentiment_chain.invoke({"text": summary})
})

# 4. 실행 예시
text = """
오늘 시험에서 만점을 받았다. 
시험 준비를 열심히 해서 결과가 좋았다.
시험이 끝난 후 친구들과 함께 축하 파티를 열었다.
시험 준비를 하면서 힘든 순간도 있었지만,
그런 순간들이 나를 더 강하게 만들어 주었다.
정말 기쁘고 노력한 보람이 있었다!
"""

result = summary_and_sentiment_chain.invoke(text)

print("📌 최종 결과:")
print(f"요약: {result['summary']}")
print(f"감정: {result['sentiment']}")

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI
from operator import itemgetter

# LLM과 출력 파서
llm = ChatOpenAI()
output_parser = StrOutputParser()

# 1. 요약 프롬프트
summarize_prompt = ChatPromptTemplate.from_template("""
다음 글을 한 문장으로 요약해줘:
{text}
""")
summarize_chain = summarize_prompt | llm | output_parser

# 2. 감정 분석 프롬프트
sentiment_prompt = ChatPromptTemplate.from_template("""
다음 글의 감정을 분석해줘. 감정은 긍정, 부정, 중립 중 하나로 답변해:
{text}
""")
sentiment_chain = sentiment_prompt | llm | output_parser

# 3. 최종 응답 프롬프트
final_prompt = ChatPromptTemplate.from_template("""
다음은 입력된 텍스트에 대한 분석 결과입니다:

요약: {summary}
감정: {sentiment}

분석을 기반으로 독자에게 전달하고 싶은 핵심 메시지를 한 문장으로 정리해주세요.
""")

# 4. 병렬 처리 체인 구성
analyze_chain = RunnableParallel({
    "summary": summarize_chain,
    "sentiment": sentiment_chain,
    "text": itemgetter("text")
}) | final_prompt | llm | output_parser

# 5. 실행 예시
input_text = {
    "text": """
오늘 시험에서 만점을 받았다. 
시험 준비를 열심히 해서 결과가 좋았다.
시험이 끝난 후 친구들과 함께 축하 파티를 열었다.
시험이 끝나고 나니 마음이 홀가분하다.
정말 기쁘고 노력한 보람이 있었다!
"""
}

result = analyze_chain.invoke(input_text)

print("📌 최종 메시지:")
print(result)
